In [ ]:
import scipy
print(scipy.__version__)

1.7.3


In [3]:
import numpy
print(numpy.__version__)

1.21.6


In [1]:
import chemprop
import sys

C:\Users\Lukas\anaconda3\envs\test_two\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unable to make new descriptors, descriptor generator requirements not properly installed
Traceback (most recent call last):
  File "C:\Users\Lukas\anaconda3\envs\test_two\lib\site-packages\descriptastorus\DescriptaStore.py", line 42, in <module>
    from .descriptors import MakeGenerator
  File "C:\Users\Lukas\anaconda3\envs\test_two\lib\site-packages\descriptastorus\descriptors\__init__.py", line 3, in <module>
    from .rdNormalizedDescriptors import *
  File "C:\Users\Lukas\anaconda3\envs\test_two\lib\site-packages\descriptastorus\descriptors\rdNormalizedDescriptors.py", line 56, in <module>
    dist = getattr(st, dist)
AttributeError: module 'scipy.stats' has no attribute 'gibrat'


AttributeError: module 'scipy.stats' has no attribute 'gibrat'

In [15]:
sys.path.append("project_resources")
from import_utils import NotebookFinder

In [16]:
sys.meta_path.append(NotebookFinder())
from project_resources.chemprop_cytochrome_P450 import *

In [4]:
models = ["linear", "KRR", "GB", "RF", "ANN"]
isozymes = ["3A4", "RLM", "HLC"]
rel_paths = {
    "3A4_source": r"project_resources/ChEMBL_3A4.csv",
    "3A4_sep": ";",
    "3A4": r"project_resources/3A4.csv",

    "RLM_source": r"project_resources/AID_1508591_datatable_all.csv",
    "RLM_sep": ",",
    "RLM": r"project_resources/RLM.csv",

    "HLC_source": r"project_resources/AID_1508603_datatable_all.csv",
    "HLC_sep": ",",
    "HLC": r"project_resources/HLC.csv",
}
# files ending in "adjusted" only contain mol IDs, smiles and t1/2 values
smiles = {}
fingerprints = {}
fp_arrays = {}
xy_splits = {}
tanimoto_sims = {}
y_test_avg_predictions = {}
stds = {}
rmsds = {}
best_models = {}
# lists of repeatedly needed and\or similar info
# dicts for storing outputs and potentially writing them into a file

In [5]:
for isozyme in isozymes:
    chemprop_data_formatting(rel_paths[f"{isozyme}_source"], isozyme, sep=rel_paths[f"{isozyme}_sep"])

3A4.csv already exists in dir
RLM.csv already exists in dir
HLC.csv already exists in dir


In [47]:
arguments = [
    "--data_path", rel_paths["3A4"],
    "--dataset_type", "regression",
    "--save_dir", "test_checkpoints"
    "--epochs", "1"
]

In [52]:
args = chemprop.args.TrainArgs().parse_args(arguments)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

TypeError: 'int' object is not subscriptable

In [8]:
arguments = [
    '--data_path', rel_paths["HLC"],
    '--dataset_type', 'regression',
    '--save_dir', 'test_checkpoints_reg2',
    '--epochs', '1',
    '--save_smiles_splits'
]

args = chemprop.args.TrainArgs().parse_args(arguments)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

Command line
python C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\ipykernel_launcher.py -f C:\Users\Lukas\AppData\Roaming\jupyter\runtime\kernel-42d837c8-d136-4119-ac64-6be062f190d1.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': False,
 'data_path': 'project_resources/HLC.csv',
 'data_weights_path': None,
 'dataset_type': 'regression',
 'depth':

ValueError: could not convert string to float: 'CC1=CC2=CC(=NC(=C2O1)NC3=CC(=CC=C3)F)C4=C(ON=C4C)C'

In [12]:
params = {
    "hidden_size": [300, 500],
    "depth": [2, 4],
    "dropout": [0, 0.2],
    "ffn_num_layers": [1, 2]
}
hyper_params = chemprop.hyperparameter_optimization.build_search_space(params, 1)
print(hyper_params)

{'hidden_size': <hyperopt.pyll.base.Apply object at 0x000001D86AD6A090>, 'depth': <hyperopt.pyll.base.Apply object at 0x000001D86ACAAC10>, 'dropout': <hyperopt.pyll.base.Apply object at 0x000001D86AC83090>, 'ffn_num_layers': <hyperopt.pyll.base.Apply object at 0x000001D86AD68150>}


In [17]:
params_list = [
    "hidden_size", [300, 500],
    "depth", [2, 4],
    "dropout", [0, 0.2],
    "ffn_num_layers", [1, 2]
]
hyper_args = chemprop.args.HyperoptArgs().parse_args(params2)
chemprop.hyperparameter_optimization.hyperopt(hyper_args)

TypeError: 'in <string>' requires string as left operand, not int

In [29]:
params2 = [
    "hidden_size", [300, 500],
    "depth", [2, 4],
    "dropout", [0, 0.2],
    "ffn_num_layers", [1, 2],
    '--config_save_path', "project_resources/hyper_params.json",
    "--data_path", rel_paths["3A4"],
    "--dataset_type", "regression"
]
params2_dict = {
    "hidden_size": [300, 500],
    "depth": [2, 4],
    "dropout": [0, 0.2],
    "ffn_num_layers": [1, 2],
    "--config_save_path": "project_resources/hyper_params.json",
    "--data_path": rel_paths["3A4"],
    "--dataset_type": "regression"
}
params3 = [
    "hidden_size", "low",
    "depth", "low",
    "dropout", "low",
    "ffn_num_layers", "low"
]
params3_dict = {
    "hidden_size": 300,
    "depth": 2,
    "dropout": "low",
    "ffn_num_layers": "low",
    "--config_save_path": "project_resources/hyper_params.json",
    "--data_path": rel_paths["3A4"],
    "--dataset_type": "regression"
}

#chemprop.args.HyperoptArgs().parse_args(params2_dict)
hyper_params = chemprop.hyperparameter_optimization.build_search_space(params3_dict, 1)
chemprop.hyperparameter_optimization.chemprop_hyperopt()

KeyError: '--config_save_path'

In [33]:
arguments = [
    '--data_path', rel_paths["3A4"],
    '--dataset_type', 'regression',
    '--save_dir', "test"
]

args = chemprop.args.TrainArgs().parse_args(arguments)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

Command line
python C:\Users\Lukas\anaconda3\envs\soc\Lib\site-packages\ipykernel_launcher.py -f C:\Users\Lukas\AppData\Roaming\jupyter\runtime\kernel-ad2d4173-6a57-46a2-99a2-820d80c9e2f6.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': False,
 'data_path': 'project_resources/3A4.csv',
 'data_weights_path': None,
 'dataset_type': 'regression',
 'depth':